In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Build Simple CNN

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128 * 1 * 1, 256),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(256, 29)
        )
    
    def forward(self, x):
        features = self.feature_extractor(x)
        return self.classifier(features)

In [3]:
model = SimpleCNN()

In [4]:
model(torch.randn(1, 3, 224, 224))

tensor([[-0.0283,  0.0933,  0.0022,  0.0991, -0.0202, -0.0183,  0.0307, -0.0095,
          0.0840,  0.0178, -0.0250, -0.0874,  0.0405,  0.1060, -0.0379, -0.1102,
          0.0620,  0.0104, -0.1138, -0.0380,  0.1143, -0.0628,  0.0169,  0.1375,
          0.0485,  0.0860,  0.1271, -0.0259, -0.0240]],
       grad_fn=<AddmmBackward>)